# Decorators

## Decorator pattern

A **decorator** is a *function* that allows one to *decorate*, i.e. run custom code *around* an existing function, passed in as a decorator’s parameter.

In [ ]:
def inspect(f):
    """Decorates a function to report its input parameters."""
    def with_inspection(*args, **kwargs):
        print("Called with args:", args, "and keyword args:", kwargs)
        return f(*args, **kwargs)
    return with_inspection

Suppose there is a function that simply adds its two parameters.

In [ ]:
def add(num, to=0):
    """Returns the result of to + num operation."""
    return to + num

print("Without decorator:", add(2, to=3))

Now, one can apply the previously declared decorator to the function and see how it works.

In [ ]:
inspected_add = inspect(add)

print("With decorator:", inspected_add(2, to=3))

It is also possible to completely replace the original function.

In [ ]:
def neg(x):
    return -x

neg = inspect(neg)

neg(3)

Replacing one function with another is such a common operation in Python that a special short-hand notation was introduced for this purpose.

In [ ]:
@inspect
def abs(x):
    return -x if x < 0 else x

abs(-5)

## One-shot side effects

It is also possible to return the *original function* from within the decorator – unchanged.

In [ ]:
registered_functions = {}

def registered(f):
    registered_functions[f.__name__] = f
    return f

@registered
def abs(x):
    return -x if x < 0 else x

@registered
def neg(x):
    return -x

print(registered_functions)

Since the decorator function `registered` is called exactly *once*, one can program a one-shot side effect to happen. This comes handly e.g. for function registration, as shown above.

## Missing docstrings and `@wraps`

Suppose that one needs to consult the documentation of the following function.

In [ ]:
def neg(x):
    """Negates input parameter and returns the result."""
    return -x

help(neg)

The result should be of no surprise to the reader. However, consider what happens if the function is decorated.

In [ ]:
@inspect
def neg(x):
    """Negates input parameter and returns the result."""
    return -x

help(neg)

The original docstring is *lost*, since the function got replaced, and the (empty) docstring of `with_inspection` function is used instead.

To fix the problem, `functools.wraps` decorator should be used for the inner function.

In [ ]:
from functools import wraps

def inspect(f):
    """Decorates a function to report its input parameters."""
    @wraps(f)   # <---
    def with_inspection(*args, **kwargs):
        print("Called with args:", args, "and keyword args:", kwargs)
        return f(*args, **kwargs)
    return with_inspection

Now the original docstring, as well as the function name, is preserved properly.

In [ ]:
@inspect
def neg(x):
    """Negates input parameter and returns the result."""
    return -x

help(neg)

neg(-3)

## Decorators with parameters \*

Suppose a `multiply` decorator is needed to post-process the original output, i.e. to return `factor * f(x)`.

Since the decorator function accepts a *function* as its sole parameter, another level of indirection is needed to handle the multiplication factor.  

In [ ]:
from functools import wraps

def multiply(factor):
    def multiply_decorator(f):
        @wraps(f)
        def wrapper(*args, **kwargs):
            return factor * f(*args, **kwargs)
        return wrapper
    return multiply_decorator

@multiply(2)
def f(x):
    return x + 3

f(5)

The code can be decomposed into 2 distinct steps:
* Outer function `multiply` can be seen as a *factory* of decorators; it produces here an anonymous `multiply_by_two` decorator from `multiply_decorator` function.
* The produced decorator replaces the original function `f` with its `wrapper` function; the wrapper modifies original output.

## Exercise

Modify the `inspect` decorator to report on the *return value* of original function, in addition to its input parameters.

In [ ]:
from functools import wraps

def inspect(f):
    """Decorates a function to report on its input and output parameters."""
    @wraps(f)
    def with_inspection(*args, **kwargs):
        print("Called with args:", args, "and keyword args:", kwargs)
        # f(*args, **kwargs)
        return
    return with_inspection

@inspect
def neg(x):
    """Negates input parameter and returns the result."""
    return -x

neg(-3)